In [ ]:
import os
import base64
import matplotlib.pyplot as plt
from mistralai import Mistral
import json
import pandas as pd
from typing import Union
import numpy as np
import secrets
import time

pd.set_option('display.max_colwidth', None)
# Load Mistral API key from environment variables
api_key = os.environ["MISTRAL_API_KEY"]

# Initialize the Mistral client
client = Mistral(api_key=api_key)

env: MISTRAL_API_KEY=ZL4LdkeCwsoIYcNx2aFDujTYgpwj2kij


In [60]:
def switch_cle():
    key = np.random.choice(["ZL4LdkeCwsoIYcNx2aFDujTYgpwj2kij","G8in7nx1BvV5jwFZeb7oYUOi9OQaKsvD","qAL87sEMQCSGKcO58179DXyH3jedFPX2","9tlWzQ18hIo6Zl9x68j99PM3J4hDgr9L"])
    client = Mistral(api_key=key)
    return client

In [85]:
def img_to_base64(image_path):
    """Input : image_path (str) : path to the image file
    Returns : image_base64 (str) : base64 encoded image"""
    
    with open(image_path, 'rb') as image_file:
        image_bytes = image_file.read()

    image_base64 = base64.b64encode(image_bytes).decode('utf-8')
    
    return image_base64

def json_to_dataframe(json_data: Union[str, dict], key: str = None) -> pd.DataFrame:
    # If json_data is a string, parse it into a dictionary
    if isinstance(json_data, str):
        json_data = json.loads(json_data)
    
    # If a key is provided, extract the list of records from the JSON object
    if key is not None:
        data = json_data[key]
    else:
        data = json_data
    
    # Convert the list of records to a pandas DataFrame
    df = pd.DataFrame(data)
    
    return df

def list_clothes(args): 
    image_base64, guide = args
    time.sleep(1)
    # Define the messages for the chat API
    messages = [
        {
            "role": "system",
            "content": "Return the answer in a JSON object with the next structure: "
                    "{\"elements\": [{\"element\": \"some name for element1\", "
                    "\"color\": \"the color of element1\", "
                    "\"fit\": \"the fit, shape of element1. Be concise.\", "
                    "\"price\": \"some number, estimated price of element1\", "
                    "\"context\": \"a word describing the occasion, mood or functionality of the piece\", "
                    "\"description\": \"a description of element1, emphasizing on the vibe of the piece and that encapsulates the precedent variables\"}, "
                    "{\"element\": \"some name for element2\", ...}]}"
        },
        {
            "role": "system",
            "content": f"You are a fashion critique, neutral and objective.\n\n {guide} \n\n You are presented with an image of an outfit, describe each of the elements thanks to your expertise "
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": f"data:image/jpeg;base64,{image_base64}"
                }
            ]
        }
    ]

    # Call the Mistral API to complete the chat
    chat_response = client.chat.complete(
        model="pixtral-12b-2409",
        messages=messages,
        temperature=0.4,
        response_format={
            "type": "json_object",
        }
    )

    # Get the content of the response
    content = chat_response.choices[0].message.content
    
    return content

def reformulate_list_clothes(args): 
    clothes = args
    time.sleep(1)
    # Define the messages for the chat API
    messages = [
        {
            "role": "system",
            "content": "Return the answer in a JSON object with the next structure: "
                    "{\"elements\": [{\"element\": \"some name for element1\", "
                    "\"color\": \"the color of element1\", "
                    "\"fit\": \"the fit, shape of element1\", "
                    "\"price\": \"some number, estimated price of element1\", "
                    "\"context\": \"a word describing the occasion, mood or functionality of the piece\", "
                    "\"description\": \"a description of element1, emphasizing on the vibe of the piece and that encapsulates the precedent variables\"}, "
                    "{\"element\": \"some name for element2\", ...}]}"
        },
        {
            "role": "system",
            "content": f"Reformulate the list of clothes, keeping the same meaning an exact same structure. Just change the wording."
        },
        {
            "role": "user",
            "content": clothes
        }
    ]

    # Call the Mistral API to complete the chat
    chat_response = client.chat.complete(
        model="pixtral-12b-2409",
        messages=messages,
        temperature=0.4,
        response_format={
            "type": "json_object",
        }
    )

    # Get the content of the response
    content = chat_response.choices[0].message.content
    
    return content

def process_critique(args):
    time.sleep(1)
    outfit, guide = args
    
    chat_response = client.chat.complete(
        model="mistral-large-latest",
        messages=[
            {"role": "system", "content": "As a 'Fashion Critique', your mission is to help relook people. \n\n {guide} \n\n You are given a description of items in an outfit. Give a critique of the outfit, outlining the general vibe, how the pieces work together and what could be improved."},
            {"role": "user", "content": outfit},
        ],
        temperature=0.2,
        max_tokens=2048
    )
    result = chat_response.choices[0].message.content

    return result

def recommend_item(args):
    client=switch_cle()
    time.sleep(1) 
    critique, guide, color_rule, piece_rule = args  
    # Define the messages for the chat API
    messages = [
        {
            "role": "system",
            "content": "Return the answer in a JSON object with the next structure: "
                    "{\"elements\": [{\"element\": \"some short name for element1\", "
                    "\"color\": \"the color of element1\", "
                    "\"fit\": \"the fit, shape of element1\", "
                    "\"price\": \"some number, estimated price of element1\", "
                    "\"context\": \"a word describing the occasion, mood or functionality of the piece\", "
                    "\"description\": \"a description of element1, emphasizing on the vibe of the piece and that encapsulates the precedent variables\"}]}"
        },
        {
            "role": "system",
            "content": f"You are a fashion critique, neutral and objective.\n\n {guide} \n\n You are presented with a critique of an outfit, describe a single element that would improve the outfit."
        },
        {
            "role": "system",
            "content": "Follow this rule when suggesting a piece of clothing: " + piece_rule
        },
        {
            "role": "system",
            "content": "Follow this rule when choosing a color: " + color_rule
        },
        {
            "role": "user",
            "content": critique
        }
    ]

    # Call the Mistral API to complete the chat
    chat_response = client.chat.complete(
        model="mistral-large-latest",
        messages=messages,
        temperature=0.4,
        response_format={
            "type": "json_object",
        }
    )

    # Get the content of the response
    content = chat_response.choices[0].message.content
    
    return content

In [3]:
desc_guide_path = "./desc_guide.txt"
critique_guide_path = "./critique_guide.txt"

with open(critique_guide_path, "rb") as f:
    critique_guide = f.read()

with open(desc_guide_path, "r") as f:
    desc_guide = f.read()

In [86]:
n_images=30
all_images = os.listdir("C:/Users/bapti/Documents/DATA/Cours/3A/Hackathon/fashion_images_dataset/")
n_images = np.random.choice(all_images, n_images, replace=False)
print(n_images)

['0368.jpg' '0017.jpg' '1849.jpg' '1012.jpg' '1924.jpg' '0431.jpg'
 '0337.jpg' '1580.jpg' '0072.jpg' '0275.jpg' '0265.jpg' '0669.jpg'
 '0349.jpg' '0622.jpg' '0540.jpg' '2038.jpg' '1488.jpg' '1969.jpg'
 '0252.jpg' '1527.jpg' '1404.jpg' '1978.jpg' '0051.jpg' '0197.jpg'
 '0160.jpg' '0620.jpg' '1137.jpg' '0377.jpg' '1174.jpg' '2008.jpg']


In [87]:
from concurrent.futures import ThreadPoolExecutor
    
args_clothes = [(img_to_base64("C:/Users/bapti/Documents/DATA/Cours/3A/Hackathon/fashion_images_dataset/"+img), desc_guide) for img in n_images]
Reformulations = []

# Using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit tasks and collect results
    Reformulations.append(list(executor.map(list_clothes, args_clothes)))

In [ ]:
# Using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit tasks and collect results
    Reformulations.append(list(executor.map(reformulate_list_clothes, Reformulations[0])))
    
# Using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit tasks and collect results
    Reformulations.append(list(executor.map(reformulate_list_clothes, Reformulations[1])))

In [88]:
Clothes_df = [json_to_dataframe(clothes, key='elements') for clothes in Reformulations[0]]
args_critiques = [('-'+'\n-'.join(clothes['description']),critique_guide) for clothes in Clothes_df]

with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit tasks and collect results
    Critiques = list(executor.map(process_critique, args_critiques))

In [ ]:
n_suggest = 1
color_rules = np.array([np.random.choice([ 
                            "complementary colors: colors that are opposite on the color wheel", 
                            "analogus colors: colors that are close on the color wheel", 
                            "accent color: one bright color that pops from the rest that are neutral", 
                            "sandwiching: layering a bright color between two neutral colors",
                            "monochromatic: using different shades of the same color to create a cohesive look",
                            "pattern mixing: combining different patterns to create a unique outfit",
                            "seasonal: using seasonal colors and pieces to create a weather-appropriate outfit",
                                        ], n_suggest, replace=False) for i in range(len(Critiques))])
        
piece_rules = np.array([np.random.choice([
                                "mixing textures: incorporating different textures to add visual interest",
                                "statement piece: building an outfit around a bold statement piece",
                                "proportion balance: suggest a piece with a fit that balances the outfit", 
                                "accessories: adding an accessory to elevate the outfit",
                                "silouhette: creating a visually interesting shape with the outfit",
                                "replacement: suggesting a piece that would replace a current piece in the outfit",
                                "layering: adding a layer to the outfit to create depth, like a coat or jacket",
                                "adding surface: suggest a pig piece with a different surface, like a shiny or matte fabric"
                                    ], n_suggest, replace=False) for i in range(len(Critiques))])
Suggestions = []

for i in range(n_suggest):
    print(i)
    args_reco = [(Critiques[j], desc_guide, color_rules[j][i], piece_rules[j][i]) for j in range(len(Critiques))]
    
    with ThreadPoolExecutor(max_workers=5) as executor:
    # Submit tasks and collect results
        Recos = list(executor.map(recommend_item, args_reco))
    
    Suggestions.append(Recos)

0


In [90]:
Desc = []
Reco = []
for i in range(n_suggest):
    for j in range(len(Critiques)):
        Desc.append(Reformulations[i][j])
        Reco.append(Suggestions[i][j])

In [91]:
random_hash = secrets.token_hex(4)
with open(f"./data/descriptions_{random_hash}.json", "w") as file:
    json.dump(Desc, file)
    
with open(f"./data/recommendations_{random_hash}.json", "w") as file:
    json.dump(Reco, file)

In [92]:
with open(f"./data/descriptions_{random_hash}.json", "r") as file:
    loaded_desc = json.load(file)
    
with open(f"./data/recommendations_{random_hash}.json", "r") as file:
    loaded_reco = json.load(file)

In [93]:
d_df=pd.concat([json_to_dataframe(loaded_desc[i],key='elements') for i in range(len(loaded_desc))])
r_df=pd.concat([json_to_dataframe(loaded_reco[i],key='elements') for i in range(len(loaded_reco))])

In [99]:
r_df.sample(10)

,element,color,fit,price,context,description
0,bracelet,gold,adjustable,150,festive,"A gold bracelet with intricate etchings that complement the dress's embroidery, adding balance and harmony to the outfit. This piece will provide a touch of elegance and continue the gold accents, continuing the festive vibe while breaking up the bold colors."
0,Neutral Handbag,Beige,Structured,150,Versatile,"A structured beige handbag with minimal detailing, perfect for balancing the vibrant colors and patterns in the outfit. Its neutral tone complements the bright pink sweater and floral maxi skirt, adding a touch of sophistication and elegance to the overall look."
0,Crossbody Bag,Neutral Beige,Small and compact with an adjustable strap,80,Casual,"A small, neutral beige crossbody bag with an adjustable strap and subtle metallic accents. This piece adds a modern, casual touch to the outfit while balancing the vibrant colors of the dress. Its smooth leather texture contrasts nicely with the dress's fabric, incorporating different textures to add visual interest, perfect for a breezy, transitional season."
0,Red Handbag,Red,Structured,150,Business Casual,"A structured red handbag to create a more cohesive look with the existing red pumps, while adding a touch of luxury and sophistication."
0,Textured Belt,Brown,Adjustable,45,Casual,"A rugged, brown leather belt with a textured finish, featuring a classic buckle and adjustable fit. This belt adds a layer of texture and interest to the outfit, complementing the smooth surfaces of the cape and jeans. It's perfect for casual settings and emphasizes a relaxed, eclectic vibe."
0,Necklace,Gold,Delicate,250,Elegant,"A delicate gold necklace with a single pendant that adds a touch of glamour and sophistication, perfect for elevating the elegance of the maroon dress and complementing the overall formal evening look."
0,Burgundy Scarf,Burgundy,One size fits all,25,Casual sophistication,"A burgundy scarf in a soft, warm fabric that adds a pop of color to the outfit while creating depth through layering. This piece sandwiches a bright color between the neutral tones of the trench coat and the tank top, enhancing the overall color palette. Its versatile nature allows it to be draped in various styles, contributing to a more harmonious and balanced silhouette."
0,Blazer,Ivory,"Slim, tailored cut with a single button closure",120,Polished,"An ivory blazer with a slim, tailored cut and single button closure, providing a polished layer that adds depth to the outfit while sandwiching the blush pink top between the black skirt and the neutral blazer."
0,denim jacket,blue,slightly oversized,45,casual,"A slightly oversized blue denim jacket, a timeless layering piece that adds depth and a casual edge to the outfit. Its blue color complements the earthy tones of the leopard print jumpsuit, creating a harmonious contrast. Perfect for a relaxed urban setting, this jacket balances the boldness of the jumpsuit while maintaining a playful, street-smart vibe."
0,Scarf,Light blue,"Loose, draped",25,"Casual, urban","A light blue scarf with a loose, draped fit, adding a pop of color to the monochromatic outfit. Perfect for casual, urban settings, this scarf elevates the look with its soft, flowing texture, creating a cohesive and stylish ensemble."
